# CNN en visión compleja
Este notebook compara dos redes convolucionales aplicadas a conjuntos de imágenes más complejas que MNIST:

- **CIFAR-10**: 10 clases generales de objetos cotidianos.
- **Traffic Sign (GTSRB)**: clasificación de señales de tráfico.

El objetivo es mostrar cómo las CNN pueden escalar a problemas reales más exigentes.

## 1. CNN en CIFAR-10

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

(X_train_cifar, y_train_cifar), (X_test_cifar, y_test_cifar) = cifar10.load_data()
X_train_cifar, X_test_cifar = X_train_cifar / 255.0, X_test_cifar / 255.0
y_train_cifar = to_categorical(y_train_cifar, 10)
y_test_cifar = to_categorical(y_test_cifar, 10)

In [ ]:
model_cifar = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [ ]:
model_cifar.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_cifar = model_cifar.fit(X_train_cifar, y_train_cifar, epochs=10, validation_data=(X_test_cifar, y_test_cifar), verbose=0)

## 2. CNN en señales de tráfico (GTSRB)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

(ds_train_gtsrb, ds_test_gtsrb), ds_info = tfds.load(
    'gtsrb',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

def preprocess_gtsrb(image, label):
    image = tf.image.resize(image, [32, 32]) / 255.0
    return image, label

ds_train_gtsrb = ds_train_gtsrb.map(preprocess_gtsrb).batch(128).prefetch(1)
ds_test_gtsrb = ds_test_gtsrb.map(preprocess_gtsrb).batch(128).prefetch(1)

model_gtsrb = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
])
model_gtsrb.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_gtsrb = model_gtsrb.fit(ds_train_gtsrb, epochs=10, validation_data=ds_test_gtsrb, verbose=0)

## 3. Comparación visual de resultados

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(history_cifar.history['val_accuracy'], label='CIFAR-10')
plt.plot(history_gtsrb.history['val_accuracy'], label='GTSRB')
plt.title('Precisión de validación por dataset (CNN)')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()